In [164]:
import pandas as pd
from pandas import DataFrame
from pandas import read_csv, pivot_table
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
from pandas import read_excel
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from helper import myOls

In [165]:
train = pd.read_excel('범죄발생구역.xlsx', index_col='Unnamed: 0')
train

,전국,아파트/연립다세대,단독주택,노상,숙박업소,유흥업소,상점,사무실,피씨방
살인기수,275,114,60,28,9,8,1,8,0
살인미수,427,123,85,105,5,24,4,13,1
강도,516,62,46,141,48,42,27,13,4
강간,5467,1616,880,158,1644,196,33,55,3
유사강간,947,240,147,57,184,55,9,26,1
...,...,...,...,...,...,...,...,...,...
협박,9187,1199,1288,1870,67,417,104,535,12
약취 유인,190,20,18,66,8,3,1,1,0
폭력행위,35686,2374,2463,12838,296,3879,207,855,75
공갈,3596,203,259,773,58,600,92,307,67


In [166]:
train.columns

Index(['전국', '아파트/연립다세대', '단독주택', '노상', '숙박업소', '유흥업소', '상점', '사무실', '피씨방'], dtype='object')

In [167]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128 entries, 살인기수 to 손괴
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   전국         128 non-null    int64
 1   아파트/연립다세대  128 non-null    int64
 2   단독주택       128 non-null    int64
 3   노상         128 non-null    int64
 4   숙박업소       128 non-null    int64
 5   유흥업소       128 non-null    int64
 6   상점         128 non-null    int64
 7   사무실        128 non-null    int64
 8   피씨방        128 non-null    int64
dtypes: int64(9)
memory usage: 10.0+ KB


In [168]:
train.describe()

,전국,아파트/연립다세대,단독주택,노상,숙박업소,유흥업소,상점,사무실,피씨방
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,18992.679688,2322.203125,1584.453125,6532.765625,419.203125,1800.093750,219.210938,445.390625,45.953125
std,36949.777329,4908.634897,3011.758870,13759.014451,605.666385,3922.651063,411.894464,786.151407,96.673479
min,187.000000,20.000000,11.000000,8.000000,4.000000,0.000000,0.000000,0.000000,0.000000
25%,549.500000,97.000000,92.000000,99.750000,39.750000,35.500000,5.000000,12.750000,1.000000
50%,2643.500000,269.000000,295.500000,249.500000,99.500000,107.000000,32.500000,54.500000,4.500000
75%,17069.750000,1768.000000,1385.000000,4995.500000,614.000000,1693.250000,252.250000,569.000000,47.000000
max,165803.000000,25179.000000,15290.000000,61152.000000,2305.000000,22772.000000,2059.000000,3784.000000,608.000000


In [169]:
names = train.columns
n = len(names)
pv = 0.05

data = []

for i in range(0, n):
    fields = names[1] + 'vs' + names[i]
    s, p = stats.pearsonr(train[names[1]], train[names[i]])
    result = p < pv

    data.append({'fields': fields, 'statistic':s, 'pvalue': p, 'result': result})

rdf = DataFrame(data)
rdf.set_index('fields', inplace=True)
rdf

,statistic,pvalue,result
fields,,,
아파트/연립다세대vs전국,0.911766,1.570084e-50,True
아파트/연립다세대vs아파트/연립다세대,1.000000,0.000000e+00,True
아파트/연립다세대vs단독주택,0.986893,1.105972e-101,True
아파트/연립다세대vs노상,0.910150,4.676855e-50,True
아파트/연립다세대vs숙박업소,0.710139,6.240115e-21,True
아파트/연립다세대vs유흥업소,0.925337,6.511608e-55,True
아파트/연립다세대vs상점,0.954894,2.680188e-68,True
아파트/연립다세대vs사무실,0.933570,5.368934e-58,True
아파트/연립다세대vs피씨방,0.917088,3.690169e-52,True


In [170]:
train.columns

Index(['전국', '아파트/연립다세대', '단독주택', '노상', '숙박업소', '유흥업소', '상점', '사무실', '피씨방'], dtype='object')

In [171]:
x = train.loc[:, ['아파트/연립다세대', '단독주택', '노상', '숙박업소', '유흥업소', '상점', '사무실', '피씨방']].values
x

array([[  114,    60,    28, ...,     1,     8,     0],
       [  123,    85,   105, ...,     4,    13,     1],
       [   62,    46,   141, ...,    27,    13,     4],
       ...,
       [ 2374,  2463, 12838, ...,   207,   855,    75],
       [  203,   259,   773, ...,    92,   307,    67],
       [ 3050,  3004, 22143, ...,   597,   798,    47]], dtype=int64)

In [172]:
import statsmodels.api as sm
results = sm.OLS(train['전국'], sm.add_constant(x)).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     전국   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     5254.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):          8.06e-148
Time:                        16:19:18   Log-Likelihood:                -1151.7
No. Observations:                 128   AIC:                             2321.
Df Residuals:                     119   BIC:                             2347.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        255.1955    246.997      1.033      0.3

In [173]:
# x = train.loc[:, ['아파트/연립다세대', '단독주택']].values
# x

In [174]:
# import statsmodels.api as sm
# results = sm.OLS(train['전국'], sm.add_constant(x)).fit()
# # add_constanst 상수항 결합(가중치 값 추정)
# results.summary()

In [175]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

In [176]:
model = model.fit(x,train['전국'])

In [177]:
y_new = model.predict(x)

In [178]:
model = LinearRegression().fit(x, train['전국'])
print(model.intercept_, model.coef_)

255.19546038997214 [ -0.0319912   -0.28572048   2.50661164   3.31682526  -0.35125162
  -6.93780834   9.27489616 -10.42077825]


In [179]:
from statsmodels.formula.api import ols
ols('전국 ~ 피씨방', data=train).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     전국   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     895.3
Date:                Mon, 25 Sep 2023   Prob (F-statistic):           4.22e-59
Time:                        16:19:18   Log-Likelihood:                -1393.4
No. Observations:                 128   AIC:                             2791.
Df Residuals:                     126   BIC:                             2797.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2547.9049   1276.081      1.997      0.048      22.579    5073.231
피씨방          357.8598     11.960     29.922      0.000     334.192     381.528
==============================================================================
Omnibus:                       31.289   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              317.559
Skew:                           0.262   Prob(JB):                     1.10e-69
Kurtosis:                      10.699   Cond. No.                         118.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [180]:
model = ols("전국 ~ 피씨방", data=train)
fit = model.fit()
tbl = fit.summary()
tbl

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     전국   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     895.3
Date:                Mon, 25 Sep 2023   Prob (F-statistic):           4.22e-59
Time:                        16:19:18   Log-Likelihood:                -1393.4
No. Observations:                 128   AIC:                             2791.
Df Residuals:                     126   BIC:                             2797.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2547.9049   1276.081      1.997      0.048      22.579    5073.231
피씨방          357.8598     11.960     29.922      0.000     334.192     381.528
==============================================================================
Omnibus:                       31.289   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              317.559
Skew:                           0.262   Prob(JB):                     1.10e-69
Kurtosis:                      10.699   Cond. No.                         118.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [181]:
train.columns

Index(['전국', '아파트/연립다세대', '단독주택', '노상', '숙박업소', '유흥업소', '상점', '사무실', '피씨방'], dtype='object')

In [182]:
x = train.loc[:, ['아파트/연립다세대', '단독주택', '노상', '숙박업소', '유흥업소', '상점', '사무실', '피씨방']].values
x

array([[  114,    60,    28, ...,     1,     8,     0],
       [  123,    85,   105, ...,     4,    13,     1],
       [   62,    46,   141, ...,    27,    13,     4],
       ...,
       [ 2374,  2463, 12838, ...,   207,   855,    75],
       [  203,   259,   773, ...,    92,   307,    67],
       [ 3050,  3004, 22143, ...,   597,   798,    47]], dtype=int64)

In [183]:
import statsmodels.api as sm
model = sm.OLS(train['전국'], train[['노상', '숙박업소', '유흥업소', '상점', '사무실']])
result = model.fit()
tbl = result.summary()
tbl

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     전국   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          1.064e+04
Date:                Mon, 25 Sep 2023   Prob (F-statistic):                   2.47e-160
Time:                        16:19:18   Log-Likelihood:                         -1153.8
No. Observations:                 128   AIC:                                      2318.
Df Residuals:                     123   BIC:                                      2332.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
노상             2.4673      0.046     53.116      0.000       2.375       2.559
숙박업소           3.4838      0.441      7.900      0.000       2.611       4.357
유흥업소          -0.5769      0.165     -3.499      0.001      -0.903      -0.251
상점            -8.5476      1.625     -5.260      0.000     -11.764      -5.331
사무실            9.3699      0.918     10.207      0.000       7.553      11.187
==============================================================================
Omnibus:                       35.882   Durbin-Watson:                   2.077
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.469
Skew:                           1.096   Prob(JB):                     1.01e-19
Kurtosis:                       6.405   Cond. No.                         145.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [184]:
import re

my = {}

for k in range(0, 3, 2):
    items = tbl.tables[k].data
    #print(items)

    for item in items:
        #print(item)
        n = len(item)

        for i in range(0, n, 2):
            key = item[i].strip()[:-1]
            value = item[i+1].strip()

            if key and value:
                my[key] = value

my

{'Dep. Variable': '전국',
 'R-squared (uncentered)': '0.998',
 'Model': 'OLS',
 'Adj. R-squared (uncentered)': '0.998',
 'Method': 'Least Squares',
 'F-statistic': '1.064e+04',
 'Date': 'Mon, 25 Sep 2023',
 'Prob (F-statistic)': '2.47e-160',
 'Time': '16:19:18',
 'Log-Likelihood': '-1153.8',
 'No. Observations': '128',
 'AIC': '2318.',
 'Df Residuals': '123',
 'BIC': '2332.',
 'Df Model': '5',
 'Covariance Type': 'nonrobust',
 'Omnibus': '35.882',
 'Durbin-Watson': '2.077',
 'Prob(Omnibus)': '0.000',
 'Jarque-Bera (JB)': '87.469',
 'Skew': '1.096',
 'Prob(JB)': '1.01e-19',
 'Kurtosis': '6.405',
 'Cond. No': '145.'}

In [185]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [186]:
for i in range(0, len(model.exog_names)):
    vif = variance_inflation_factor(model.exog, i)

    if vif < 10:
        print("%s의 VIF: %f (good)" % (model.exog_names[i], vif))
    else:
        print("%s의 VIF: %f (bad)" % (model.exog_names[i], vif))

노상의 VIF: 15.462096 (bad)
숙박업소의 VIF: 3.262764 (good)
유흥업소의 VIF: 15.636961 (bad)
상점의 VIF: 17.764561 (bad)
사무실의 VIF: 21.260394 (bad)


In [187]:
my['variables'] = []

for i, v in enumerate(tbl.tables[1].data):
    if i == 0:
        continue

    # 변수의 이름
    name = v[0].strip()
    # 변수의 이름 목록
    name_list = list(model.exog_names)
    # 변수의 이름 목록에서 현재 변수가 몇 번째 항목인지 찾기 
    j = name_list.index(name)

    vif = 0

    # 0번째인 Intercept는 제외
    if j >= 0:
        vif = variance_inflation_factor(model.exog, j)

    my['variables'].append({
        "name": name,
        "coef": v[1].strip(),
        "std err": v[2].strip(),
        "t": v[3].strip(),
        "P-value": v[4].strip(),
        "Beta": 0,
        "VIF": vif,
    })

my

{'Dep. Variable': '전국',
 'R-squared (uncentered)': '0.998',
 'Model': 'OLS',
 'Adj. R-squared (uncentered)': '0.998',
 'Method': 'Least Squares',
 'F-statistic': '1.064e+04',
 'Date': 'Mon, 25 Sep 2023',
 'Prob (F-statistic)': '2.47e-160',
 'Time': '16:19:18',
 'Log-Likelihood': '-1153.8',
 'No. Observations': '128',
 'AIC': '2318.',
 'Df Residuals': '123',
 'BIC': '2332.',
 'Df Model': '5',
 'Covariance Type': 'nonrobust',
 'Omnibus': '35.882',
 'Durbin-Watson': '2.077',
 'Prob(Omnibus)': '0.000',
 'Jarque-Bera (JB)': '87.469',
 'Skew': '1.096',
 'Prob(JB)': '1.01e-19',
 'Kurtosis': '6.405',
 'Cond. No': '145.',
 'variables': [{'name': '노상',
   'coef': '2.4673',
   'std err': '0.046',
   't': '53.116',
   'P-value': '0.000',
   'Beta': 0,
   'VIF': 15.462096326971219},
  {'name': '숙박업소',
   'coef': '3.4838',
   'std err': '0.441',
   't': '7.900',
   'P-value': '0.000',
   'Beta': 0,
   'VIF': 3.262764349920089},
  {'name': '유흥업소',
   'coef': '-0.5769',
   'std err': '0.165',
   't': 

In [188]:
mylist = []

for i in my['variables']:
    if i['name'] == 'Intercept':
        continue

    item = {
        "독립변수": i['name'],
        "B": i['coef'],
        "표준오차": i['std err'],
        "β": i['Beta'],
        "t": "%s*" % i['t'],
        "유의확률" : i['P-value'],
        "VIF": i["VIF"]
    }

    mylist.append(item)

df = DataFrame(mylist, index=['전국','전국','전국','전국','전국'])
df.index.name = '종속변수'
df

,독립변수,B,표준오차,β,t,유의확률,VIF
종속변수,,,,,,,
전국,노상,2.4673,0.046,0,53.116*,0.000,15.462096
전국,숙박업소,3.4838,0.441,0,7.900*,0.000,3.262764
전국,유흥업소,-0.5769,0.165,0,-3.499*,0.001,15.636961
전국,상점,-8.5476,1.625,0,-5.260*,0.000,17.764561
전국,사무실,9.3699,0.918,0,10.207*,0.000,21.260394
